In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [2]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [3]:
df = pd.read_csv('Silver Star Properties Houston holdings - Sheet1.csv')

In [5]:
df.at[0,'Address'] = ('11211 KATY FREEWAY, HOUSTON, TEXAS')
df.at[1,'Address'] = ('1400 BROADFIELD BOULEVARD, HOUSTON, TEXAS')

In [7]:
df['geo_address'] = df['Address']

In [8]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [9]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [50]:
# df['image_links'] = ''
# links = []
# for url in df['story_link']:
#     try:
#         page = requests.get(url)
#         soup = BeautifulSoup(page.content, 'html.parser')
#         images = soup.find('img',{'class':'FeaturedBlock_image__GyCn6'})
#         images = images['src']
#         links.append(images)
#     except:
#         links.append('no_image')

# df['image_links'] = links

In [10]:
# df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
# df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
# df['remaining_desc']

# df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
# df['description_link']

In [11]:
df.columns

Index(['Address', 'Appraisal value', 'Description', 'geo_address', 'geocoded',
       'lat', 'lon'],
      dtype='object')

In [12]:
def popup_html(row):
    i = row
    address = df['Address'].iloc[i]
    AV = df['Appraisal value'].iloc[i]
    description = df['Description'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Appraisal Value: </strong>{}'''.format(AV) + '''<br>
    <strong>Description: </strong>{}'''.format(description) + '''<br>
    </html>
    '''
    return html

In [17]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Silver Star Properties Houston holdings')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html,height=300)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [58]:
m.save('index.html')

## Map URL Snagger

In [59]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Hialeah-development
